In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
import env_configurations
sess = None

robot lib not fouund

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'BipedalWalker-v2'#'Pendulum-v0'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
obs_space, action_space = env_configurations.get_obs_and_action_spaces(env_name)
print(action_space.low)
print(action_space.high)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[-1. -1. -1. -1.]
[1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-06-07 11:27:57,936	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-07_11-27-57_5933/logs.
2019-06-07 11:27:58,049	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:28661 to respond...
2019-06-07 11:27:58,173	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:29611 to respond...
2019-06-07 11:27:58,178	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-06-07 11:27:58,225	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-07_11-27-57_5933/logs.
2019-06-07 11:27:58,229	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:28661',
 'object_store_address': '/tmp/ray/session_2019-06-07_11-27-57_5933/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-06-07_11-27-57_5933/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_continuous import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

a2c_config = a2c_games_configurations.bipedalwalker_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#a2c_config = a2c_games_configurations.pendulum_config
#a2c_config = a2c_games_configurations.quadrupped_config
#a2c_config = a2c_games_configurations.mountain_car_cont_config
agent = A2CAgent(sess,'car', obs_space, False, action_space, a2c_config)
agent.restore('nn/latest')
#agent.train()


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-06-07 11:27:59,737	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/latest


In [6]:
ray.shutdown()

In [9]:
import numpy as np
#agent.save('nn/latest')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
def evaluate(env,t_max=5000):
    rewards = []
    #env._max_episode_steps = 5000
    env.batch_mode = True
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
env = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
env.close()




/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
reset
[[-0.74637234  0.35445604  0.21643904 -0.7129681 ]]
[[ 0.02127649 -0.9211977   0.49142334 -0.61317575]]
[[-0.6330911  -0.24651605  0.43272257 -0.19413656]]
[[-0.4792707  -0.40586329  0.43041107  0.02468108]]
[[-0.5248351  -0.6785711   0.41260105  0.73896587]]
[[-0.5343057   0.30150235  0.425782    0.78891736]]
[[-0.45297432  0.33061168  0.3310726   0.45677692]]
[[-0.47080588  0.25506124  0.19230123  0.24386558]]
[[-0.47120708  0.22585386  0.06404461 -0.06754266]]
[[-0.35716626  0.10382891  0.10046023 -0.4686419 ]]
[[ 0.1814318  -0.837207    0.0399166  -0.90084195]]
[[-0.00874575 -0.4549672   0.27401292 -0.579787  ]]
[[ 0.22861354 -0.4964361   0.3750612  -0.3329493 ]]
[[ 0.42414835 -0.385583    0.16651337 -0.14663446]]
[[-0.12735328 -0.12214725  0.24195099  0.32665628]]
[[-0.35251546  0

[[ 0.10280396  0.99735314 -0.15436623 -0.00828461]]
[[ 0.43489158  0.9709778  -0.08894507  0.03215849]]
[[ 0.31978926  1.         -0.13702893 -0.27131402]]
[[ 0.8041342  -0.45986295  0.463256   -0.9043199 ]]
[[ 0.36586672 -0.2041499   0.30181798 -0.5839299 ]]
[[ 0.52521396 -0.23277125  0.35747322 -0.37952194]]
[[ 0.39009658 -0.17829727  0.48027837 -0.25628915]]
[[ 0.3669191  -0.25490323  0.5792067  -0.04544601]]
[[ 0.0240608   0.400943    0.47434485 -0.26020166]]
[[ 0.6291888  -0.14037843  0.28233624 -0.47880477]]
[[ 0.24210532 -0.13701198  0.19924986 -0.08012076]]
[[ 0.16146745 -0.02845755  0.3832602   0.2507693 ]]
[[0.0809624  0.3554441  0.46850142 0.26182723]]
[[-0.62230486  0.8882981   0.5985211   0.41183233]]
[[0.02477338 1.         0.21531942 0.5920644 ]]
[[-0.03373285  0.95409346  0.06891984  0.40014854]]
[[0.0480342  0.93219066 0.21710825 0.20648679]]
[[ 0.07841563  1.         -0.12933129  0.08503184]]
[[ 3.6242303e-01  1.0000000e+00 -2.3903698e-04 -2.5764260e-01]]
[[ 0.7703373

KeyboardInterrupt: 

In [8]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

TypeError: 'int' object is not subscriptable

In [ ]:
import gym
gym.envs.registry.all()